In [1]:
import pandas as pd
import os

In [2]:
# This is also in my repo data folder
raw_df = pd.read_csv("https://raw.githubusercontent.com/NIVANorge/oslomod_phase3_teotil/refs/heads/main/data/results_summary.csv")
raw_df

,Scenario,Område,Kilde,Parameter,Verdi (tonn)
0,Baseline,001,Bakgrunn,DIN,106.293181
1,Baseline,001,Bakgrunn,SS,404.177361
2,Baseline,001,Bakgrunn,TDP,0.787351
3,Baseline,001,Bakgrunn,TOC,7407.268113
4,Baseline,001,Bakgrunn,TON,221.985402
...,...,...,...,...,...
2715,Scenario_B,Hele Oslofjord,Spredt avløp,TOC,1693.351996
2716,Scenario_B,Hele Oslofjord,Spredt avløp,TON,173.047267
2717,Scenario_B,Hele Oslofjord,Spredt avløp,TOTN,722.918192
2718,Scenario_B,Hele Oslofjord,Spredt avløp,TOTP,56.507919


In [7]:
# Add pristine scenario
print(raw_df['Kilde'].unique())

# Step 1 & 2: Filter Baseline rows and exclude Kilde == 'Bakgrunn'
baseline_filtered = raw_df[(raw_df['Scenario'] == 'Baseline') & (raw_df['Kilde'] == 'Bakgrunn')]

# Step 3: Copy the filtered rows
pristine_rows = baseline_filtered.copy()

# Step 4: Update Scenario to 'Pristine'
pristine_rows['Scenario'] = 'Pristine'

# Step 5: Add these rows back to the original DataFrame
df_extended = pd.concat([raw_df, pristine_rows], ignore_index=True)

df_extended

['Bakgrunn' 'Bebygd' 'Industri' 'Jordbruk' 'Kommunalt avløp'
 'Spredt avløp' 'Akvakultur']


,Scenario,Område,Kilde,Parameter,Verdi (tonn)
0,Baseline,001,Bakgrunn,DIN,106.293181
1,Baseline,001,Bakgrunn,SS,404.177361
2,Baseline,001,Bakgrunn,TDP,0.787351
3,Baseline,001,Bakgrunn,TOC,7407.268113
4,Baseline,001,Bakgrunn,TON,221.985402
...,...,...,...,...,...
2875,Pristine,Hele Oslofjord,Bakgrunn,TOC,198161.535443
2876,Pristine,Hele Oslofjord,Bakgrunn,TON,7855.348401
2877,Pristine,Hele Oslofjord,Bakgrunn,TOTN,11412.737016
2878,Pristine,Hele Oslofjord,Bakgrunn,TOTP,120.412186


In [8]:
df = df_extended.groupby(['Scenario', 'Område', 'Parameter']).sum()
df = df.drop('Kilde', axis=1).reset_index()
df

,Scenario,Område,Parameter,Verdi (tonn)
0,Baseline,001,DIN,651.771194
1,Baseline,001,SS,3674.260039
2,Baseline,001,TDP,10.280051
3,Baseline,001,TOC,8217.373138
4,Baseline,001,TON,427.974122
...,...,...,...,...
635,Scenario_B,Ytre Oslofjord,TOC,206394.091068
636,Scenario_B,Ytre Oslofjord,TON,11230.068645
637,Scenario_B,Ytre Oslofjord,TOTN,26239.714978
638,Scenario_B,Ytre Oslofjord,TOTP,479.295350


In [9]:
# Create a dictionary of DataFrames, one per Scenario
dfs_by_scenario = {}

for scenario, group in df.groupby('Scenario'):
    # Pivot the group: Område as index, Parameter as columns, Verdi (tonn) as values
    pivoted = group.pivot(index='Område', columns='Parameter', values='Verdi (tonn)')
    dfs_by_scenario[scenario] = pivoted

    fpath = f"../data/t3_summary/t3_flux_to_of_{scenario}.csv"
    pivoted.to_csv(fpath)

dfs_by_scenario['Baseline'].tail()

Parameter,DIN,SS,TDP,TOC,TON,TOTN,TOTP,TPP
Område,,,,,,,,
016,2155.597849,9184.814556,28.838106,20134.380530,1439.935639,3595.533488,53.654759,24.816653
017,240.631872,1473.517764,4.236652,5581.787683,307.246243,547.878115,7.705405,3.468752
Hele Oslofjord,23155.460894,331860.259069,339.065942,224326.810897,13794.506792,36949.967686,924.509616,585.443674
Indre Oslofjord,1823.029524,15994.814305,44.148365,10588.975647,782.481021,2605.510545,97.568947,53.420582
Ytre Oslofjord,21332.431370,315865.444765,294.917577,213737.835250,13012.025771,34344.457141,826.940669,532.023092


In [10]:
dfs_by_scenario['Pristine'].tail()

Parameter,DIN,SS,TDP,TOC,TON,TOTN,TOTP,TPP
Område,,,,,,,,
016,459.343105,844.870817,7.724485,18313.484502,1074.625274,1533.968379,10.506334,2.781849
017,119.962282,374.173825,1.259993,5226.414590,263.895083,383.857365,2.283610,1.023617
Hele Oslofjord,3557.388615,26317.740686,55.510328,198161.535443,7855.348401,11412.737016,120.412186,64.901858
Indre Oslofjord,85.940876,795.046736,1.084679,7020.632817,160.763994,246.704871,3.804322,2.719643
Ytre Oslofjord,3471.447739,25522.693950,54.425649,191140.902626,7694.584407,11166.032145,116.607864,62.182215
